# Commodity Storage Model

**Randall Romero Aguilar, PhD**

This demo is based on the original Matlab demo accompanying the  <a href="https://mitpress.mit.edu/books/applied-computational-economics-and-finance">Computational Economics and Finance</a> 2001 textbook by Mario Miranda and Paul Fackler.

Original (Matlab) CompEcon file: **demode05.m**

Running this file requires the Python version of CompEcon. This can be installed with pip by running

    !pip install compecon --upgrade

<i>Last updated: 2021-Oct-05</i>
<hr>

## About

Solve

\begin{align*}
\dot{s} &= -p^{-\eta}\\
\dot{p} &= rp+\kappa
\end{align*}

where

* $s$: stocks
* $p$: price



## FORMULATION

In [1]:
#from compecon import jacobian, ODE

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Velocity Function


In [2]:
r = 0.1  # interest rate
κ = 0.5  # unit cost of storage
η = 5    # demand elasticity

def f(x):
    s, p = x
    return [-p**(-η),  r*p + κ]

### Time Horizon

In [3]:
T = 1
xlabels  = ['$s$','$p$']


### Boundary Conditions

In [4]:
s0, sT  = 1, 0  # initial and terminal stocks
bx = [1, 1]     # boundary variables
bt = [0, T]     # boundary times
bv = [s0, sT]   # boundary values

In [18]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from compecon import BasisChebyshev, BasisSpline, NLP, jacobian, gridmake

class ODE:
    def __init__(self, f, T, bv, *params, labels=None):
        self.f = lambda x: f(x, *params)
        self.T = T
        self.bv = bv
        self._d = len(self.bv)
        self.fsol = None
        self.xspx = None

        if labels is not None:
            assert len(labels) ==self._d, "ERROR, number of labels must equal number of variables in system."
            self.labels = labels
        else:
            self.labels = [f'$y_{j}$' for j in range(self._d)]

    def solve_collocation(self, *, n=100, bt=None, bx=None, btype='cheb', y=None, nf=10):
        if bt is None:
            bt = np.zeros_like(self.bv)
        if bx is None:
            bx = np.arange(len(self.bv))

        basis = BasisChebyshev if btype.lower() == 'cheb' else BasisSpline
        T = self.T

        # compute collocation nodes
        t = basis(n - 1, 0, T).nodes

        # Approximation structure
        self.fsol = basis(n, 0, T, l=self.labels, labels=['Time'])  # falta inicializar los coeficientes

        if y:
            self.fsol.y += y
        print(f'{self.fsol.c=}')

        # residual function for nonlinear problem formulation
        def residual(c):
            # reshape coefficient vector
            self.fsol.c = c.reshape(self._d, n)

            # compute residuals at nodal times
            x = self.fsol(t)
            dx = self.fsol(t, 1)
            r = dx - self.f(x)

            # compute residuals at boundaries
            x = self.fsol(bt)
            b = np.array([x[j, bx[j]] - self.bv[j] for j in range(self._d)])
            b = np.atleast_2d(b).T
            return np.c_[r, b].flatten()

        # Solve the nonlinear system
        self.fsol.c = NLP(residual).broyden(x0=self.fsol.c.flatten(), show=True).reshape(self._d, n)

        # Compute solution at plotting nodes
        if nf > 0:
            m = int(nf) * n
            t = np.linspace(0, T, m)
        else:
            t = t.flatten()

        x = self.fsol(t)

        # Compute residual
        dx = self.fsol(t, 1)
        resid = dx - self.f(x)

        self.x = pd.DataFrame(x.T, index=t, columns=self.labels)
        self.resid = pd.DataFrame(resid.T, index=t, columns=self.labels)

## SOLVE ODE USING COLLOCATION METHOD

In [20]:
n = 15    # number of basis functions

problem = ODE(f, T, bv, labels=xlabels)
problem.solve_collocation(n=n, bt=bt, bx=bx, nf=10, y=1.0)

#problem.x['$p$'].plot()
#c = zeros(n,2); c(1,:) = 1;
#[x,t,res] = odecol(f,bv,T,n,bt,bx,[],c);

self.fsol.c=array([[ 1.0000e+00, -1.1102e-16,  1.1102e-16, -1.1102e-16,  4.1633e-17, -2.7756e-17,  5.5511e-17, -1.3878e-17,
        -2.7756e-17, -1.3878e-17,  1.2490e-16, -3.4694e-17,  2.7756e-17,  2.4286e-17,  7.1124e-17],
       [ 1.0000e+00, -1.1102e-16,  1.1102e-16, -1.1102e-16,  4.1633e-17, -2.7756e-17,  5.5511e-17, -1.3878e-17,
        -2.7756e-17, -1.3878e-17,  1.2490e-16, -3.4694e-17,  2.7756e-17,  2.4286e-17,  7.1124e-17]])
Solving nonlinear equations by Broyden's method
it    bstep  change
--------------------
   0     6  9.93e-01
   1     7  9.92e-01
   2     6  9.89e-01
   3     7  9.88e-01
   4     6  9.87e-01
   5     7  9.86e-01
   6     6  9.86e-01
   7     7  9.85e-01
   8     7  9.81e-01
   9     7  9.81e-01
  10     7  9.76e-01
  11     8  9.75e-01
  12     7  9.70e-01
  13     8  9.70e-01
  14     7  9.63e-01
  15     9  9.64e-01
  16     7  9.63e-01
  17     1  7.69e+05
  18     2  7.53e+04
  19     2  1.18e+06
  20     0  3.74e+05
  21     0  1.93e+05
  22     0  